In [0]:
# -------------------- Configuration --------------------

import requests
import json
from datetime import date, timedelta

ACCOUNT_NAME = 'datapr0ject'
TIERS = ['bronze', 'silver', 'gold']

API_KEY = dbutils.secrets.get(scope="sports-api-secrets", key="thesportsdb_key")
if not API_KEY:
    raise ValueError("API Key not found. Please check secret scope setup.")

season_year = 2025
sport_name = "Fighting"

start_date = date.today() - timedelta(days=1)

In [0]:
# -------------------- ADLS Paths Setup --------------------

def get_adls_paths(account_name, tiers):
    return {tier: f'abfss://{tier}@{account_name}.dfs.core.windows.net/' for tier in tiers}

adls_paths = get_adls_paths(ACCOUNT_NAME, TIERS)

bronze_adls = adls_paths['bronze']
silver_adls = adls_paths['silver']
gold_adls = adls_paths['gold']

dbutils.fs.ls(bronze_adls)
dbutils.fs.ls(silver_adls)
dbutils.fs.ls(gold_adls)

In [0]:
# -------------------- Known API Sport IDs --------------------
sports_ids = {
    "fighting": "4443",  # UFC
    "soccer": "4328",
    "basketball": "4387",
    "baseball": "4424",
    "american_football": "4391",
    "rugby": "4464",
    "motorsport": "4370",
    "cycling": "4406",
    "darts": "4415"
}

In [0]:
# -------------------- League ID Resolver --------------------

def get_league_id(sport_name, default_id=None):
    try:
        return sports_ids[sport_name.lower()]
    except KeyError:
        print(f"[ERROR] Sport '{sport_name}' not found in local mapping.")
        print("Please refer to https://www.thesportsdb.com/documentation for valid sport names.")
        return default_id

In [0]:
# -------------------- API Call + Save --------------------

def fetch_and_save_season_events(sport_id, season_year):
    url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/eventsseason.php?id={sport_id}&s={season_year}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json().get("events", [])
        if not data:
            print("No data returned for the specified year or sport.")
            return

        file_path = f"{bronze_adls}/{start_date}_{sport_id}_season_data.json"
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API: {e}")

In [0]:
# -------------------- Execute ETL --------------------

sport_id = get_league_id(sport_name)
fetch_and_save_season_events(sport_id, season_year)